In [14]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.5.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0 pyspark-shell'

In [15]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession \
        .builder \
        .appName("test") \
        .getOrCreate()

In [16]:
streaming_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "10.0.0.10:9092") \
    .option("subscribe", "crypto_rates") \
    .option("startingOffsets", "earliest") \
    .load()

In [17]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json
from pyspark.sql.types import StructType, StringType, DoubleType, LongType

# Define the schema for JSON messages
schema = StructType() \
    .add("id", StringType()) \
    .add("symbol", StringType()) \
    .add("currencySymbol", StringType()) \
    .add("type", StringType()) \
    .add("rateUsd", DoubleType()) \
    .add("timestamp", LongType())

# Parse JSON messages from Kafka topic
parsed_df = streaming_df.selectExpr("CAST(value AS STRING)") \
    .select(from_json("value", schema).alias("data")).select("data.*")



In [18]:
query = parsed_df  \
    .writeStream \
    .format("parquet") \
    .option("path", "output_parquet") \
    .option("checkpointLocation", "checkpoint_dir_output_parquet") \

# Wait for the query to terminate
query.awaitTermination()

AttributeError: 'DataStreamWriter' object has no attribute 'awaitTermination'